In this notebook, we are going to perform top down market analysis to determine which stocks should be bought. Before we consider buying individual stocks, we have to analyze the overall market condition using dual momentum theory. Based on the dual momentum theory, we only invest in assets when their prices are going up or in uptrend.

In [1]:
import momentum
import pyticker
import dividend
import pandas as pd
import datetime as dt

## Global Macro Momentum

We can compute previous 12 months' momentums of some of top ETFs in different asset classes to observe the overall market movements. I've written helper functions to compute equally weighted momentum in momentum.py which is imported above. Below, we used the avearge of previous 1 month return, 3 months return, 6 months return and 12 months return to measure the momentum.

In [2]:
global_macro = ['SPY', 'QQQ', 'TLT', 'IEF', 'GLD', 'DBC']
start_date = dt.datetime(1970,1,1)
end_date = dt.datetime.today()
momentum_df = momentum.calculate_equal_weight_momentum(global_macro, start_date, end_date, [1,3,6,12])
momentum_df

,Name,1M_Return,3M_Return,6M_Return,12M_Return,EW_MOMENTUM
Symbol,,,,,,
QQQ,"Invesco QQQ Trust, Series 1",0.017168,0.018465,0.152146,0.684186,0.217991
DBC,Invesco DB Commodity Index Trac,-0.007173,0.129932,0.271822,0.476444,0.217757
SPY,SPDR S&P 500,0.045399,0.063518,0.192400,0.562311,0.215907
GLD,SPDR Gold Trust,-0.011433,-0.103162,-0.096883,0.080446,-0.032758
IEF,iShares 7-10 Year Treasury Bond,-0.023857,-0.057335,-0.069430,-0.061508,-0.053033
TLT,iShares 20+ Year Treasury Bond,-0.052453,-0.139224,-0.164905,-0.167392,-0.130993


Based on the calcution above, we can see that QQQ has been the strongest market over the last year with 21.7% positive returns. Next, DBC and SPY have also around 21% returns. In contrast, Gold and US Bonds have negative returns over the last year. Clearly, equity markets have been much stronger than Gold and US Bonds markets.

The underlying assumption is that if we start investing today, we should consider buying assets in equities or DBC rather than Gold or US Bonds as equities and DBC have higher momentum scores.

## US Sector Momentum

### 1. Extract Sector Table from etfdb.com

In [4]:
url = 'https://etfdb.com/etfs/sector/'
df = pd.read_html(url)[0].drop([11], axis=0)

sectors = list(df['Sector'])
sectors = [x.lower() for x in sectors]
sectors = [x.replace(' ', '-') for x in sectors]
sectors = [x.replace('discretionary', 'discretionaries') for x in sectors]
#sectors

### 2. Retrieve Top ETFs from Each Sector

Assumption here is that ETFs with the largest AUMs in each sector contains major companies in each sector, and their performance are reflected on the performance of ETFs. Of course, performance of individual stocks may vary from the overall performance of ETFs, but it's good to measure the overall sentiment of sector by analyzing the price performance of top ETFs.

In [5]:
top_etfs = {}
for s in sectors:
    sector_url = url + s + '/'
    sector_data = pd.read_html(sector_url)[0]
    etf_symbol = sector_data.loc[0, 'Symbol']
    etf_name = sector_data.loc[0, 'ETF Name']
    etf_industry = sector_data.loc[0, 'Industry']
    etf_aum = sector_data.loc[0, 'Total Assets ($MM)']
    top_etfs[s] = {}
    top_etfs[s]['symbol'] = etf_symbol
    top_etfs[s]['name'] = etf_name
    top_etfs[s]['industry'] = etf_industry
    top_etfs[s]['aum'] = etf_aum

In [8]:
sector_data = {
    'Symbol': [],
    'Sector': [],
    'Industry': [],
    'AUM': []
}

for etf in top_etfs.keys():
    symbol = top_etfs[etf]['symbol']
    sector = etf
    industry = top_etfs[etf]['industry']
    aum = top_etfs[etf]['aum']

    sector_data['Symbol'].append(symbol)
    sector_data['Sector'].append(sector)
    sector_data['Industry'].append(industry)
    sector_data['AUM'].append(aum)

### 3. Sector Momentum Calculation

In [15]:
sector_momentum = momentum.calculate_equal_weight_momentum(sector_data['Symbol'], start_date, end_date, [1,3,6,12])

AttributeError: 'DataFrame' object has no attribute 'set_values'

In [16]:
sector_df = pd.DataFrame(sector_data)
sector_df.set_index('Symbol', inplace=True)
sector_df['EW_MOMENTUM'] = sector_momentum['EW_MOMENTUM']
sector_df.sort_values(by='EW_MOMENTUM', ascending=False)

,Sector,Industry,AUM,EW_MOMENTUM
Symbol,,,,
XLE,energy,Oil & Gas Exploration & Production,"$22,340.30",0.448858
XLF,financials,Broad Financials,"$39,667.60",0.329591
XLI,industrials,Broad Industrials,"$20,586.80",0.296919
VOX,telecom,Broad Telecom,"$3,754.81",0.274553
XLY,consumer-discretionaries,Broad Consumer Discretionary,"$20,816.30",0.241553
QQQ,technology,Broad Technology,"$167,873.00",0.217991
VNQ,real-estate,Broad Real Estate,"$37,261.60",0.174088
XLV,healthcare,Broad Healthcare,"$25,617.30",0.131741
XLP,consumer-staples,Broad Consumer Staples,"$10,658.70",0.118145


Based on the momentum calculation above, Oil & Gas sector has the highest momentum score with 44.8% positive returns and Gold industry has the lowest momentum score with 4.96% positive returns.

## US Individual Stocks

S&P 500 -> 20 Years Dividend Payout -> Dividend Growth greater than 8% -> uptrend

In [ ]:
equity_universe = ['DOW JONES']
symbols = []
for equity in equity_universe:
    data = pyticker.get_symbols_by_index(equity)
    for d in data:
        if d not in symbols:
            symbols.append(d)

In [ ]:
threshold = 20

dividend_symbols = []

for symbol in symbols:
    if fundamentals.get_consecutive_dividend_payout_history(symbol, threshold) != None:
        dividend_symbols.append(symbol)


In [ ]:
print(fundamentals.get_consecutive_dividend_payout_history('CRM', 20))

In [ ]:
dividend_symbols

In [ ]:
data_dict = {'Symbol': [], 'Dividend Growth': [], 'Dividend Yield': []}

yrs = 15

for symbol in dividend_symbols:
    print(symbol)
    data_dict['Symbol'].append(symbol)
    data_dict['Dividend Growth'].append(fundamentals.calcualte_avg_dividend_growth(symbol, yrs))
    data_dict['Dividend Yield'].append(fundamentals.calculate_current_dividend_yield(symbol))   

In [ ]:
data_dict

In [ ]:
df = pd.DataFrame(data_dict)
df.sort_values(by=['Dividend Growth', 'Dividend Yield'], ascending=False, inplace=True)

In [ ]:
df

In [ ]:
df = df[(df['Dividend Growth'] >= 0.05) & (df['Dividend Yield'] > 0.01)]
df

In [ ]:
df.set_index('Symbol', inplace=True)

In [ ]:
df

In [ ]:
df = df.sort_values(by='Dividend Yield', ascending=False)
df

In [ ]:
mom = momentum.calculate_equal_weight_momentum(list(df.index), start_date, end_date, [1,3,6,12])

In [ ]:
mom

In [ ]:
df['EW_MOMENTUM'] = mom['EW_MOMENTUM']

In [ ]:
df

In [ ]:
df = df.sort_values(by='EW_MOMENTUM', ascending=False)
df

In [ ]:
dt.datetime.strptime('2020-10-01', '%Y-%m-%d')